In [1]:
import cv2
import time
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import numpy as np

In [2]:
from cvzone.HandTrackingModule import HandDetector

In [3]:
from pycaw.pycaw import AudioUtilities , IAudioEndpointVolume

In [4]:
cap = cv2.VideoCapture(0)

In [5]:
detector = HandDetector(detectionCon=0.8,maxHands=1)
cap.set(3,1280)
cap.set(4,720)

True

In [6]:
previous_time = 0
cap = cv2.VideoCapture(0)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volume_range = volume.GetVolumeRange()
min_volume = volume_range[0]
max_volume = volume_range[1]
volume.SetMasterVolumeLevel(0, None)
vol=0
while True:
    status, photo = cap.read()
    hands, photo = detector.findHands(photo)
    current_time = time.time()
    fps = 1 / (current_time - previous_time)
    previous_time = current_time
    cv2.putText(photo, f'FPS : {int(fps)}', (40, 70), cv2.FONT_HERSHEY_COMPLEX, 3, (50, 50, 50), 3)
    if len(hands) != 0:
        lmList = hands[0]['lmList']
        length, info, photo = detector.findDistance(lmList[4][:2], lmList[8][:2], photo)
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cv2.circle(photo, (x1, y1), 30, (50, 50, 50), cv2.FILLED)
        cv2.circle(photo, (x2, y2), 30, (50, 50, 50), cv2.FILLED)
        vol = np.interp(length, [50, 300], [min_volume, max_volume])
        volume.SetMasterVolumeLevel(vol, None)
    cv2.rectangle(photo, (50, 150), (85, 400), (0, 50, 0), 3)
    cv2.rectangle(photo, (50, int(np.interp(vol, [min_volume, max_volume], [400, 150]))), (85, 400), (0, 50, 0), cv2.FILLED)
    if len(hands) != 0 and length < 50:
        cv2.circle(photo, (x1, y1), 30, (0, 255, 0), cv2.FILLED)
    
    cv2.imshow("myphoto", photo)
    if cv2.waitKey(1) & 0xFF == 13:  # Exit loop on Enter key
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Aitha Nushith\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
cap.release()